# ***Funding Analysis***

In [1]:
%tensorflow_version 1.x
from google.colab import files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import to_categorical

TensorFlow 1.x selected.


Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd drive/Shareddrives

/content/drive/Shareddrives


In [4]:
cd "TAMIDS'22"

/content/drive/Shareddrives/TAMIDS'22


In [5]:
grantDf = pd.read_csv('Dimensions-Grant-2022-03-12_21-16-43.csv', header = 1)
grantDf.columns

Index(['Rank', 'Grant ID', 'Grant Number', 'Title', 'Title translated',
       'Abstract', 'Abstract translated', 'Funding Amount', 'Currency',
       'Funding Amount in USD', 'Start Date', 'Start Year', 'End Date',
       'End Year', 'Researchers', 'Research Organization - original',
       'Research Organization - standardized', 'GRID ID',
       'City of Research organization', 'State of Research organization',
       'Country of Research organization', 'Funder', 'Funder Group',
       'Funder Country', 'Resulting publications', 'Source Linkout',
       'Dimensions URL', 'FOR (ANZSRC) Categories', 'RCDC Categories',
       'HRCS HC Categories', 'HRCS RAC Categories', 'ICRP Cancer Types',
       'ICRP CSO Categories', 'Units of Assessment',
       'Sustainable Development Goals'],
      dtype='object')

In [6]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.stem.wordnet import WordNetLemmatizer

custom_stopwords = ['Texas','A','&',"M",'College', 'Station', 'Center', 'School', 'Institute', 'Division', 'Principal', 'Investigator', 'Grant', 'project', 'research', 'ha', 'approach', 'student', 'impact', 'result', 'work', 'use', 'method', 'area', 'effort', 'PhD']

grantDf = grantDf.fillna(0)

grantDf['tokens'] = grantDf.apply(lambda row: nltk.word_tokenize(str(row['Abstract translated'])), axis=1)
grantDf['lemmas'] = grantDf['tokens'].apply(lambda lst:[WordNetLemmatizer().lemmatize(word) for word in lst])
grantDf['lemmas'] = grantDf['lemmas'].apply(lambda x: ' '.join(y for y in x if y not in (stop) and y not in (custom_stopwords)))

for index_label, row_series in grantDf.iterrows():
  nouns = [] #empty to array to hold all nouns
  if str(grantDf.at[index_label , 'Abstract translated']) == "0":
    sentences = grantDf.at[index_label , 'Title translated']
  else:
    sentences = grantDf.at[index_label , 'lemmas']
  for word,pos in nltk.pos_tag(nltk.word_tokenize(str(sentences))):
      if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'):
          nouns.append(word.lower())
  grantDf.at[index_label , 'Abstract'] = list(set(nouns))
  grantDf.at[index_label, 'Funding Amount in USD'] = float(grantDf.at[index_label, 'Funding Amount in USD']) / float((grantDf.at[index_label, 'End Year'] - grantDf.at[index_label, 'Start Year']) + 1)
grantDf

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Rank,Grant ID,Grant Number,Title,Title translated,Abstract,Abstract translated,Funding Amount,Currency,Funding Amount in USD,...,FOR (ANZSRC) Categories,RCDC Categories,HRCS HC Categories,HRCS RAC Categories,ICRP Cancer Types,ICRP CSO Categories,Units of Assessment,Sustainable Development Goals,tokens,lemmas
0,100,grant.9967334,2140664,CAREER: Heat Semigroups and Strichartz Estimat...,CAREER: Heat Semigroups and Strichartz Estimat...,"[assumption, wave, environment, theory, physic...",This award is funded in whole or in part under...,424657,USD,70776,...,0102 Applied Mathematics; 01 Mathematical Scie...,0,0,0,0,0,B10 Mathematical Sciences,0,"[This, award, is, funded, in, whole, or, in, p...",This award funded whole part American Rescue P...
1,100,grant.9752183,2125985,SCC-IRG Track1: ADVANCED LEARNING FOR ENERGY R...,SCC-IRG Track1: ADVANCED LEARNING FOR ENERGY R...,"[b, option, machine, measure, office, fundamen...",This NSF S&CC project aims to predict the Stat...,1498793,USD,299758,...,1117 Public Health and Health Services; 11 Med...,0,0,0,0,0,B11 Computer Science and Informatics,13 Climate Action,"[This, NSF, S, &, CC, project, aims, to, predi...",This NSF S CC aim predict State Risk ( SoR ) e...
2,100,grant.9971039,2144973,CAREER: Dislocation-level Understanding of She...,CAREER: Dislocation-level Understanding of She...,"[level, delay, crack, electron, solid, stokes,...",PART 1: NON-TECHNICAL SUMMARY Magnesium (Mg) i...,464494,USD,77415,...,0912 Materials Engineering; 09 Engineering,0,0,0,0,0,B12 Engineering,0,"[PART, 1, :, NON-TECHNICAL, SUMMARY, Magnesium...",PART 1 : NON-TECHNICAL SUMMARY Magnesium ( Mg ...
3,100,grant.9982325,2142177,"CAREER: Past, Present, and Future of the Peatl...","CAREER: Past, Present, and Future of the Peatl...","[future, timing, test, initiation, holocene, e...",Tropical ecosystems are subject to some of the...,816374,USD,136062,...,0602 Ecology; 0502 Environmental Science and M...,0,0,0,0,0,C14 Geography and Environmental Studies,13 Climate Action,"[Tropical, ecosystems, are, subject, to, some,...",Tropical ecosystem subject highest rate land-u...
4,100,grant.9964747,2145628,CAREER: Integrating river hydrology across sca...,CAREER: Integrating river hydrology across sca...,"[assumption, product, goal, driver, criterion,...",Rivers are natural hotspots for greenhouse gas...,535242,USD,89207,...,0406 Physical Geography and Environmental Geos...,0,0,0,0,0,C14 Geography and Environmental Studies,13 Climate Action,"[Rivers, are, natural, hotspots, for, greenhou...",Rivers natural hotspot greenhouse gas emission...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,100,grant.3339396,8501724,Mathematical Sciences: Algebraic Cycles on Sur...,Mathematical Sciences: Algebraic Cycles on Sur...,"[algebraic, sciences, cycles, values, surfaces...",0,31000,USD,10333,...,0,0,0,0,0,0,0,0,[0],0
4996,100,grant.3338933,8500363,Purchase of a High Resolution Fourier Transfor...,Purchase of a High Resolution Fourier Transfor...,"[photometer, transform, purchase, fourier, spe...",0,175000,USD,87500,...,0,0,0,0,0,0,0,0,[0],0
4997,100,grant.3346213,8550784,Graduate Research Fellowships Support Grant,Graduate Research Fellowships Support Grant,"[fellowships, grant, support, graduate, research]",0,46500,USD,7750,...,0,0,0,0,0,0,0,0,[0],0
4998,100,grant.3341827,8509821,Warm Core Ring Interdisciplinary Synthesis Pro...,Warm Core Ring Interdisciplinary Synthesis Pro...,"[core, warm, radium, rings, interdisciplinary,...",0,37157,USD,12385,...,0,0,0,0,0,0,0,0,[0],0


In [7]:
buzzwordDf = grantDf[["Grant ID", "Grant Number", "Abstract", "Funding Amount", "Funding Amount in USD", "Start Year", "End Year", "FOR (ANZSRC) Categories", "Resulting publications"]]
buzzwordDf = buzzwordDf.sort_values(by=['Funding Amount in USD'], ascending=False)
buzzwordDf.rename(columns = {'Abstract':'Abstract Keywords'}, inplace = True)
buzzwordDf.rename(columns = {'Funding Amount':'Original Funding Amount'}, inplace = True)
buzzwordDf.rename(columns = {'Funding Amount in USD':'Funding Amount per year'}, inplace = True)
buzzwordDf.rename(columns = {'FOR (ANZSRC) Categories':'Departments'}, inplace = True)
buzzwordDf

,Grant ID,Grant Number,Abstract Keywords,Original Funding Amount,Funding Amount per year,Start Year,End Year,Departments,Resulting publications
561,grant.4320258,DE-FG02-93ER40773,"[base, focus, science, how, question, equipmen...",78661000,26220333,2020,2022.0,"0202 Atomic, Molecular, Nuclear, Particle and ...",pub.1092302380; pub.1103354615; pub.1083780985...
2076,grant.3957202,EP/M013294/1,"[user, centre, device, manufacturing/r, transf...",35513856,9746534,2014,2019.0,0299 Other Physical Sciences; 02 Physical Scie...,pub.1060766780; pub.1092035477; pub.1122311154...
1223,grant.7056592,1648451,"[burden, device, environment, future, blood, m...",20315000,3385833,2017,2022.0,0903 Biomedical Engineering; 09 Engineering,pub.1137410711; pub.1120983632; pub.1112995952...
4431,grant.3460940,9802942,"[institution, estudios, undergraduate, //www.n...",17460462,2494351,1998,2004.0,1608 Sociology; 16 Studies in Human Society,pub.1093437577; pub.1094806986; pub.1093569157...
2372,grant.3946882,2014-31100-06048,"[tx, hatch, act]",7125137,2375045,2013,2015.0,0,0
...,...,...,...,...,...,...,...,...,...
546,grant.10000112,0,"[pilot, mobility, conduct, town, +, industry, ...",494855,-245,2020,0.0,1117 Public Health and Health Services; 11 Med...,0
1792,grant.7051720,RP160589,"[perturbation, step, environment, alterations,...",890840,-442,2015,0.0,0601 Biochemistry and Cell Biology; 06 Biologi...,pub.1132949899; pub.1084060973; pub.1134851705...
2030,grant.7051642,RP150454,"[level, interferons, dna, synthes, therapy, ta...",900000,-446,2015,0.0,1107 Immunology; 0605 Microbiology; 0601 Bioch...,pub.1014178261; pub.1115134691; pub.1105943169...
1976,grant.9425921,PP150089,"[ses, increasing, border, mammography, equipme...",1500000,-744,2015,0.0,1117 Public Health and Health Services; 11 Med...,pub.1132335303; pub.1023512338; pub.1129530362


In [8]:
buzzwordDf_trunc = buzzwordDf[["Abstract Keywords", "Funding Amount per year", "Departments"]]
for index_label, row_series in buzzwordDf_trunc.iterrows():
  buzzwordDf_trunc.at[index_label , 'Departments'] = str(row_series['Departments']).split(";")
for index_label, row_series in buzzwordDf_trunc.iterrows():
  buzzwordDf_trunc.at[index_label , 'Departments'] = [x.strip() for x in row_series['Departments']]

In [9]:
for index_label, row_series in buzzwordDf_trunc.iterrows():
  valList = row_series['Departments']
  valNewList = []
  for val in valList:
    if int(val[:2]) == 1:
      valNewList.append("Mathematics")
    elif int(val[:2]) == 2:
      valNewList.append("Physics")
    elif int(val[:2]) == 3:
      valNewList.append("Chemistry")
    elif int(val[:2]) == 4:
      valNewList.append("Geology and Geophysics")
    elif int(val[:2]) == 5:
      valNewList.append("Environmental Science")
    elif int(val[:2]) == 6:
      valNewList.append("Biology and Genetics")
    elif int(val[:2]) == 7:
      valNewList.append("Agriculture and Crop Sciences")
    elif int(val[:2]) == 8:
      valNewList.append("AI and Computer Science")
    elif int(val[:2]) == 9:
      valNewList.append("Engineering")
    elif int(val[:2]) == 10:
      valNewList.append("Biotechnology")
    elif int(val[:2]) == 11:
      valNewList.append("Medical and Health Sciences")
    elif int(val[:2]) == 12:
      valNewList.append("Urban and Regional Planning")
    elif int(val[:2]) == 13:
      valNewList.append("Education")
    elif int(val[:2]) == 14:
      valNewList.append("Economics")
    elif int(val[:2]) == 15:
      valNewList.append("Commerce, Management, Tourism and Services")
    elif int(val[:2]) == 16:
      valNewList.append("Studies in Human Society")
    elif int(val[:2]) == 17:
      valNewList.append("Psychology and Cognitive Sciences")
    elif int(val[:2]) == 18:
      valNewList.append("Law and Legal Studies")
    elif int(val[:2]) == 20:
      valNewList.append("Language, Communication and Culture")
    elif int(val[:2]) == 21:
      valNewList.append("History and Archaeology")
    elif int(val[:2]) == 22:
      valNewList.append("Philosophy and Religious Studies")
    
  buzzwordDf_trunc.at[index_label , 'Departments'] = list(set(valNewList))

In [10]:
#Abstract noun exploding happens here
buzzwordDf_exploded = buzzwordDf_trunc.explode('Abstract Keywords').reset_index(drop=True)
buzzwordDf_exploded['Abstract Keywords'] = buzzwordDf_exploded['Abstract Keywords'].str.strip()

In [11]:
#Department explode happens here
buzzwordDf_exploded = buzzwordDf_exploded.explode('Departments').reset_index(drop=True)
buzzwordDf_exploded['Departments'] = buzzwordDf_exploded['Departments'].str.strip()

In [12]:
depList = list(set(buzzwordDf_exploded['Departments']))[1:]
grouped_df = buzzwordDf_exploded.groupby('Departments')
dataframe_collection = {}

for dep in depList:
  dataframe_collection[dep] = grouped_df.get_group(dep)
  dataframe_collection[dep] = pd.DataFrame(dataframe_collection[dep].groupby('Abstract Keywords')['Funding Amount per year'].sum()).sort_values(by='Funding Amount per year', ascending=False).head(100)

In [13]:
dataframe_collection['AI and Computer Science']

,Funding Amount per year
Abstract Keywords,
system,40596150
data,30818085
support,28474727
development,26435570
model,24953199
...,...
device,7603717
management,7586225
health,7531125


# ***Publication and Citation Analysis***

In [14]:
cd Dataset/

/content/drive/Shareddrives/TAMIDS'22/Dataset


In [15]:
df = pd.read_csv('citation_altmetric.csv')
df['publication_score'] = pd.Series([1 for x in range(len(df.index))])
for index_label, row_series in df.iterrows():
  df.at[index_label , 'Author\'s department'] = str(row_series['Author\'s department']).split("||")

In [16]:
#Department explode happens here
df_exploded = df.explode('Author\'s department').reset_index(drop=True)
df_exploded['Author\'s department'] = df_exploded['Author\'s department'].str.strip()

pub_df = df_exploded[["Publication title", "citation_score", "publication_score", "Author\'s department"]]

In [17]:
for index_label, row_series in pub_df.iterrows():
  dep = row_series['Author\'s department']
  histList = ['History', 'University Libraries']
  bioList = ['Biochemistry and Biophysics','Biology','Biomedical Sciences','Center For Biomedical Informatics','Center for Statistical Bioinformatics','College of Veterinary Medicine and Biomedical Sciences','Ecology and Conservation Biology', 'Epidemiology and Biostatistics','Institute of Biosciences and Technology','Marine Biology','Plant Pathology and Microbiology','Veterinary Integrative Biosciences', 'Veterinary Pathobiology']
  geoList = ['Geology and Geophysics']
  agriList = ['Agricultural Economics', 'Agricultural Leadership, Education, and Communications','College of Agriculture and Life Sciences','Norman E. Borlaug Institute for International Agriculture', 'Agricultural Leadership, Education, and Communications','Center for Sport Management Research and Education','College of Education and Human Development', 'Educational Administration and Human Resource Development', 'Educational Psychology']
  engList = ['Aerospace Engineering', 'Biological and Agricultural Engineering','Biomedical Engineering', 'Chemical Engineering', 'Chemical Engineering (Qatar)','Civil Engineering', 'Computer Science and Engineering', 'Electrical and Computer Engineering', 'Engineering Education Research Taskforce','Engineering Medicine', 'Engineering Technology and Industrial Distribution','Industrial and Systems Engineering', 'Institute for Engineering Education and Innovation','Marine Engineering Technology','Materials Science and Engineering', 'Mechanical Engineering', 'Multidisciplinary Engineering', 'Nuclear Engineering', 'Ocean Engineering','Petroleum Engineering']
  mathList = ['Mathematics']
  phyList = ['Physics and Astronomy']
  chemList = ['Chemistry']

  if any(str(dep) in s for s in histList):
    pub_df.at[index_label , 'Author\'s department'] = "History and Archaeology"
  elif any(str(dep) in s for s in bioList):
    pub_df.at[index_label , 'Author\'s department'] = "Biology and Genetics"
  elif any(str(dep) in s for s in geoList):
    pub_df.at[index_label , 'Author\'s department'] = "Geology and Geophysics"
  elif any(str(dep) in s for s in agriList):
    pub_df.at[index_label , 'Author\'s department'] = "Agriculture and Crop Sciences"
  elif any(str(dep) in s for s in engList):
    pub_df.at[index_label , 'Author\'s department'] = "Engineering"
  elif any(str(dep) in s for s in mathList):
    pub_df.at[index_label , 'Author\'s department'] = "Mathematics"
  elif any(str(dep) in s for s in phyList):
    pub_df.at[index_label , 'Author\'s department'] = "Physics"
  elif any(str(dep) in s for s in chemList):
    pub_df.at[index_label , 'Author\'s department'] = "Chemistry"

In [18]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.stem.wordnet import WordNetLemmatizer

custom_stopwords = ['Texas','A','&',"M",'College', 'Station', 'Center', 'School', 'Institute', 'Division', 'Principal', 'Investigator', 'Grant', 'project', 'research', 'ha', 'approach', 'student', 'impact', 'result', 'work', 'use', 'method', 'area', 'effort', 'PhD']

pub_df = pub_df.fillna(0)

pub_df['tokens'] = pub_df.apply(lambda row: nltk.word_tokenize(str(row['Publication title'])), axis=1)
pub_df['lemmas'] = pub_df['tokens'].apply(lambda lst:[WordNetLemmatizer().lemmatize(word) for word in lst])
pub_df['lemmas'] = pub_df['lemmas'].apply(lambda x: ' '.join(y for y in x if y not in (stop) and y not in (custom_stopwords)))

for index_label, row_series in pub_df.iterrows():
  nouns = [] #empty to array to hold all nouns
  sentences = pub_df.at[index_label , 'lemmas']
  for word,pos in nltk.pos_tag(nltk.word_tokenize(str(sentences))):
      if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'):
          nouns.append(word.lower())
  pub_df.at[index_label , 'Publication title'] = list(set(nouns))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
pub_df = pub_df[pub_df.citation_score != 0]
pub_df = pub_df[["Publication title", "citation_score", "Author\'s department", "publication_score"]]

In [20]:
#Abstract noun exploding happens here
pub_df_exploded = pub_df.explode('Publication title').reset_index(drop=True)
pub_df_exploded['Publication title'] = pub_df_exploded['Publication title'].str.strip()
pub_df_exploded = pub_df_exploded.sort_values(by=['citation_score'], ascending=False)
pub_df_exploded.rename(columns = {'Publication title':'Abstract Keywords'}, inplace = True)

In [21]:
grouped_df = pub_df_exploded.groupby('Author\'s department')

In [22]:
pub_dataframe_collection = {}

for dep in depList:
  try:
    pub_dataframe_collection[dep] = grouped_df.get_group(dep)
    pub_dataframe_collection[dep] = pd.DataFrame(pub_dataframe_collection[dep].groupby('Abstract Keywords')['publication_score'].sum()).sort_values(by='publication_score', ascending=False).head(100)
  except:
    continue

In [23]:
cit_dataframe_collection = {}

for dep in depList:
  try:
    cit_dataframe_collection[dep] = grouped_df.get_group(dep)
    cit_dataframe_collection[dep] = pd.DataFrame(cit_dataframe_collection[dep].groupby('Abstract Keywords')['citation_score'].sum()).sort_values(by='citation_score', ascending=False).head(100)
  except:
    continue

In [24]:
import pandas as pd

for dep in depList:
  try:
    dataframe_collection[dep] = pd.merge(dataframe_collection[dep], pub_dataframe_collection[dep], on=['Abstract Keywords'])
  except:
    continue

for dep in depList:
  try:
    dataframe_collection[dep] = pd.merge(dataframe_collection[dep], cit_dataframe_collection[dep], on=['Abstract Keywords'])
  except:
    continue

In [25]:
dept_mean_var = {'History and Archaeology': [0.305,0.103], 'Chemistry': [0.775, 0.626],'Agriculture and Crop Sciences':[1.514,7.803],'Studies in Human Society':[1.464,10.578],
                  'Education':[1.336,4.603],'Environmental Science': [0.434,0.108],'Commerce, Management, Tourism and Services':[0.759,1.294],'Economics':[0.98,6.4],
                 'Law and Legal Studies':[0.51,0.302], 'Philosophy and Religious Studies':[0.278,0.026],'Engineering':[0.788,2.712],'Biology and Genetics':[1.223,3.11] ,
                 'AI and Computer Science': [0.749,1.236], 'Psychology and Cognitive Sciences':[1.272,5.838],'Physics': [2.176,290.906],
                 'Mathematics':[0.485,0.191], 'Language, Communication and Culture':[0.333,0.088], 'Geology and Geophysics':[0.76,1.812],
                 'Medical and Health Sciences':[1.183,2.586],'Urban and Regional Planning':[1.345,10.258], 'Biotechnology':[0.601, 0.218] }

In [26]:
for key in dataframe_collection.keys():
  dataframe_collection[key]['Funding Normalized'] = (dataframe_collection[key]['Funding Amount per year']-10**(5)*dept_mean_var[key][0])/(dept_mean_var[key][1]*10**(5))
  try:
    dataframe_collection[key]['Impact Score'] = dataframe_collection[key]['Funding Normalized']*dataframe_collection[key]['publication_score']*dataframe_collection[key]['citation_score']/(100*dataframe_collection[key]['citation_score'].sum(axis=0))
  except:
    continue

In [61]:
import plotly.express as px

for dep in depList:
  try:
    pd.DataFrame(dataframe_collection[dep]).to_csv(str(dep)+".csv")
    df = pd.read_csv(str(dep)+".csv")

    fig = px.scatter_3d(df, x = 'publication_score', 
                      y = 'citation_score', 
                      z = 'Impact Score',
                      color = 'Abstract Keywords', 
                      size='Funding Normalized',
                      size_max = 50, 
                      opacity = 0.75)
    
    fig.write_html(str(dep)+".html")
  except:
    continue

#pd.DataFrame(dataframe_collection['Chemistry']).to_csv('myfile.csv')

In [52]:
df = pd.read_csv('myfile.csv')
df

,Abstract Keywords,Funding Amount per year,publication_score,citation_score,Funding Normalized,Impact Score
0,study,13644919,7,145,216.731933,0.608865
1,development,13385159,7,173,212.582412,0.712531
2,structure,10692993,8,310,169.576565,1.163991
3,synthesis,9051701,10,331,143.357843,1.313353
4,effect,8489466,4,200,134.376454,0.297540
5,design,7741567,6,203,122.429185,0.412728
6,interaction,7713906,7,195,121.987316,0.460871
7,metal,7565371,8,170,119.614553,0.450251
8,mechanism,6738297,5,295,106.402508,0.434386
9,water,4763343,6,151,74.853722,0.187704


In [58]:
import plotly.express as px
  
fig = px.scatter_3d(df, x = 'publication_score', 
                    y = 'citation_score', 
                    z = 'Impact Score',
                    color = 'Abstract Keywords', 
                    size='Funding Normalized',
                    size_max = 50, 
                    opacity = 0.75)
  
fig.show()
fig.write_html(".html")

In [62]:
ls

'Agriculture and Crop Sciences.csv'
'Agriculture and Crop Sciences.html'
'AI and Computer Science.csv'
'Biology and Genetics.csv'
'Biology and Genetics.html'
 Biotechnology.csv
 Chemistry.csv
 Chemistry.html
 citation_altmetric.csv
 citation_altmetric.gsheet
'Commerce, Management, Tourism and Services.csv'
 Economics.csv
 Education.csv
 Engineering.csv
 Engineering.html
'Environmental Science.csv'
'Geology and Geophysics.csv'
'Geology and Geophysics.html'
'History and Archaeology.csv'
'History and Archaeology.html'
'Language, Communication and Culture.csv'
'Law and Legal Studies.csv'
 Mathematics.csv
 Mathematics.html
'Medical and Health Sciences.csv'
 myfile.csv
'Philosophy and Religious Studies.csv'
 Physics.csv
 Physics.html
'Psychology and Cognitive Sciences.csv'
'Studies in Human Society.csv'
'Urban and Regional Planning.csv'
